In [13]:
import plotly.graph_objs as go
import pandas as pd
from plotly.offline import iplot
import numpy as np
import statistics 
import plotly.express as px
from numpy.lib import recfunctions as rfn

In [14]:
e7_t09_rawCount_data = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T09_logNorm.csv.gz", sep=",", index_col=0,low_memory=False) #pandas DataFrame
e7_t10_rawCount_data = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T10_logNorm.csv.gz", sep=",", index_col=0,low_memory=False) #pandas DataFrame
e7_t17_rawCount_data = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T17_logNorm.csv.gz", sep=",", index_col=0,low_memory=False) #pandas DataFrame
e7_t18_rawCount_data = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T18_logNorm.csv.gz", sep=",", index_col=0,low_memory=False) #pandas DataFrame
e7_t21_rawCount_data = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T21_logNorm.csv.gz", sep=",", index_col=0,low_memory=False) #pandas DataFrame
e7_t22_rawCount_data = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T22_logNorm.csv.gz", sep=",", index_col=0,low_memory=False) #pandas DataFrame

cell_types = pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_E7_cell_type.txt", sep="\t") #pandas DataFrame for cell types

In [15]:
cell_types_list = ['Alpha','Beta','Delta','Gamma','Epsilon'] #the cell types of interest for this work
cell_types = cell_types[cell_types['CellType'].isin(cell_types_list)] # parsing sampele type list, to only contain the cell types of interest
cell_types.reset_index(drop=True, inplace=True)

updated_list = []
for t in cell_types['SampleID'].values:
    if t in ['T09', 'T10']:
        updated_list.append('2Hr')
    elif t in ['T17','T18']:
        updated_list.append('12Hr')
    elif t in ['T21','T22']:
        updated_list.append('24Hr')
    else:
        updated_list.append('NA')

        
cell_types['TimePoint'] = updated_list

In [16]:
# cell_types
# Data dict
sample_dict = {
    'T09': pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T09_logNorm.csv.gz", sep=",", index_col=0,low_memory=False), #pandas DataFrame,
    'T10': pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T10_logNorm.csv.gz", sep=",", index_col=0,low_memory=False), #pandas DataFrame
    'T17': pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T17_logNorm.csv.gz", sep=",", index_col=0,low_memory=False),
    'T18': pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T18_logNorm.csv.gz", sep=",", index_col=0,low_memory=False),
    'T21': pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T21_logNorm.csv.gz", sep=",", index_col=0,low_memory=False),
    'T22': pd.read_csv("/Users/krilowcn/Desktop/Projects/GSIS/data/normalizedData/200613_E7_logNorm/200613_T22_logNorm.csv.gz", sep=",", index_col=0,low_memory=False),
}

In [17]:
cells = pd.DataFrame([], columns = ['gene', 'mean_expr_1', 'mean_expr_2', 'cell_type', 'time_point'])

for cell_type in np.unique(cell_types['CellType']):
    if cell_type == "unknown":
        continue
    sub_cells = cell_types[cell_types.CellType == cell_type]
    for time_point in np.unique(cell_types['TimePoint']):
        ## To do -- deal with High / low glucose treatment
        sampls = np.unique(cell_types[cell_types.TimePoint == time_point]['SampleID'])
#         Get sample IDs
        counts_1 = sample_dict.get(sampls[0])
        counts_2 = sample_dict.get(sampls[1])
        if counts_1 is None or counts_2 is None:
            continue
        
        barcodes_1 = sub_cells[sub_cells.SampleID == sampls[0]]['CellID'].values
        barcodes_2 = sub_cells[sub_cells.SampleID == sampls[1]]['CellID'].values
        means_1 = np.mean(counts_1.loc[:, barcodes_1],axis=1) 
        means_2 = np.mean(counts_2.loc[:, barcodes_2],axis=1)
        joint_index = np.intersect1d(means_1.index.values, means_2.index.values)

        cells = cells.append( #look at the gene name for R1 and R2, output that and compare to see if they match or not*
            pd.DataFrame({
                'gene': joint_index,
                'mean_expr_1': means_1.loc[joint_index].values,
                'mean_expr_2': means_2.loc[joint_index].values,
                'cell_type': [cell_type] * len(joint_index),
                'time_point': [time_point] * len(joint_index)  
            }))

In [18]:
list_ordering = ['2Hr','12Hr','24Hr']  
cells["time_point"] = pd.Categorical(cells["time_point"], categories=list_ordering) 


In [21]:
import plotnine as plt9
from plotnine import *
from plotnine.data import *

%matplotlib inline

plt9.options.figure_size = (22,10)

p=plt9.ggplot(cells, plt9.aes(
    x='mean_expr_1', 
    y = 'mean_expr_2')) + plt9.geom_point(
        alpha=0.50) + geom_abline(intercept=0, 
    slope=1) + plt9.facet_grid('cell_type ~ time_point') + labs(title="Counts comparison between technical replicates (E07-High Glucose)",
                                                                x="Replicate-1",y="Replicate-2",size="Gene",color="Gene") + theme_classic(base_family = "Helvetica")+geom_point(color='darkgreen')


p.save(filename = 'E07_high_glucose.png', height=5, width=5, units = 'in', dpi=1000)

/Users/krilowcn/anaconda3/envs/BINF/lib/python3.8/site-packages/plotnine/ggplot.py:727: PlotnineWarning: Saving 5 x 5 in image.
/Users/krilowcn/anaconda3/envs/BINF/lib/python3.8/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: E07_high_glucose.png
/Users/krilowcn/anaconda3/envs/BINF/lib/python3.8/site-packages/plotnine/layer.py:467: PlotnineWarning: geom_point : Removed 19640 rows containing missing values.
/Users/krilowcn/anaconda3/envs/BINF/lib/python3.8/site-packages/plotnine/layer.py:467: PlotnineWarning: geom_point : Removed 19640 rows containing missing values.
